In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

In [2]:
chromedriver_location = "chrome_driver/chromedriver"

In [3]:
# xPaths to each required search element in the field. 
district_drop_down = '//*[@id="courtId"]'
date_field = '//*[@id="inputVO.startDate"]'
#seven_day_search_radio = '//*[@id="sevenDayRadio"]'
search_by_attorney_radio = '//*[@id="searchAttorneyRadio"]'
mobar_input = '//*[@id="inputVO.mobarNumber"]'
submit_button = '//*[@id="findButton"]'

In [4]:
# create an instance of the webdriver and load the inital calendar search page
driver = webdriver.Chrome(chromedriver_location)
driver.get('https://www.courts.mo.gov/casenet/cases/calendarSearch.do')

# store the district dropdown in an object using Select
courtDistrictDrp = Select(driver.find_element_by_xpath(district_drop_down))
courtDistrictDrp.select_by_value("CT11") # 11th District Saint Charles

# date
driver.find_element_by_xpath(date_field).send_keys("8/10/2021")
# 7 day search results
#driver.find_element_by_xpath(seven_day_search_radio).click()
# search by attorney
driver.find_element_by_xpath(search_by_attorney_radio).click()
# MOBAR number field 
driver.find_element_by_xpath(mobar_input).send_keys("71855")

# without the pause, the inputs are too fast for casenet to handle
time.sleep(1) 

# submit 
driver.find_element_by_xpath(submit_button).click()



In [5]:
dates = driver.find_elements_by_xpath('//td[contains(text(), ", 20")]')

In [5]:
current_date = driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td/table/tbody/tr[1]/td[2]').text

In [6]:
current_date

' Monday, August 16, 2021'

In [7]:
i=1
data = []
while driver.find_elements_by_xpath(f'//td[@class="td{i}"]'):
    data.append(driver.find_elements_by_xpath(f'//td[@class="td{i}"]'))
    i += 1

In [8]:
raw_text = []
for d in data:
    for i in d:
        raw_text.append(i.text)

In [9]:
def slice_per(source, step):
    return [source[i::step] for i in range(step)]

In [10]:
organized_data = slice_per(raw_text, 9)

In [11]:
len(organized_data)

9

In [12]:
organized_data

[['1911-CR01190-01  ',
  '1911-CR01457-01  ',
  '1811-CR03256-01  ',
  '1911-CR02239-01  ',
  '1911-CR01436-01  ',
  '2011-CR02440-01  ',
  '2011-CR00723-01  '],
 ['ST V JEFFREY CALVIN HILL ',
  'ST V SEAN D. GWIN ',
  'ST V JULIANNE MARIE POPE ',
  'ST V BRIAN KEITH BILLINGSLEY ',
  'ST V MARLA A.M. NEIL ',
  'ST V CARLOS L. SPANN ',
  'ST V JULIANNE MARIE POPE '],
 ['8:30 AM ',
  '8:30 AM ',
  '9:00 AM ',
  '1:30 PM ',
  '8:30 AM ',
  '8:30 AM ',
  '9:00 AM '],
 ['1 OF 1', '1 OF 1', '1 OF 1', '1 OF 1', '1 OF 1', '1 OF 1', '1 OF 1'],
 [' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['Plea Hearing ',
  'Disposition Hearing ',
  'Sentencing Hearing ',
  'Pre-trial Conference ',
  'Disposition Hearing ',
  'Arraignment ',
  'Plea Hearing '],
 ['Event Text:  ',
  'Event Text:  ',
  'Event Text:  ',
  'Event Text:  ',
  'Event Text: ',
  'Event Text: ',
  'Event Text: '],
 ['Location: DIVISION 4 COURTROOM St Charles Circuit Div',
  'Location: DIVISION 4 COURTROOM St Charles Circuit Div',
  'Location

In [13]:
df = pd.DataFrame({
    "date" : current_date,
    "case_num" : organized_data[0],
    "case_style" : organized_data[1],
    "time": organized_data[2],
    "days": organized_data[3],
    "event": organized_data[5],
    "location": organized_data[7],
    "judge": organized_data[8]
})

In [14]:
df.head()

,date,case_num,case_style,time,days,event,location,judge
0,"Monday, August 16, 2021",1911-CR01190-01,ST V JEFFREY CALVIN HILL,8:30 AM,1 OF 1,Plea Hearing,Location: DIVISION 4 COURTROOM St Charles Circ...,Judge/Commissioner: MICHAEL JAMES FAGRAS
1,"Monday, August 16, 2021",1911-CR01457-01,ST V SEAN D. GWIN,8:30 AM,1 OF 1,Disposition Hearing,Location: DIVISION 4 COURTROOM St Charles Circ...,Judge/Commissioner: MICHAEL JAMES FAGRAS
2,"Monday, August 16, 2021",1811-CR03256-01,ST V JULIANNE MARIE POPE,9:00 AM,1 OF 1,Sentencing Hearing,Location: DIVISION 7 COURTROOM St Charles Circ...,Judge/Commissioner: DANIEL G PELIKAN
3,"Monday, August 16, 2021",1911-CR02239-01,ST V BRIAN KEITH BILLINGSLEY,1:30 PM,1 OF 1,Pre-trial Conference,Location: DIVISION 7 COURTROOM St Charles Circ...,Judge/Commissioner: DANIEL G PELIKAN
4,"Monday, August 16, 2021",1911-CR01436-01,ST V MARLA A.M. NEIL,8:30 AM,1 OF 1,Disposition Hearing,Location: DIVISION 4 COURTROOM St Charles Circ...,Judge/Commissioner: MICHAEL JAMES FAGRAS
